In [3]:
from  qblockchain import QBlockchain



ModuleNotFoundError: No module named 'qblockchain'

In [4]:
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

# display current supported backends
print(provider.backends())



[<IBMBackend('ibmq_belem')>, <IBMBackend('simulator_extended_stabilizer')>, <IBMBackend('ibmq_manila')>, <IBMBackend('ibm_nairobi')>, <IBMBackend('ibm_lagos')>, <IBMBackend('ibmq_jakarta')>, <IBMBackend('ibmq_quito')>, <IBMBackend('ibmq_lima')>, <IBMBackend('simulator_statevector')>, <IBMBackend('simulator_stabilizer')>, <IBMBackend('ibm_perth')>, <IBMBackend('ibmq_qasm_simulator')>, <IBMBackend('simulator_mps')>]


In [5]:
# get IBM's simulator backend
simulator_backend = provider.get_backend('ibmq_qasm_simulator')
print(simulator_backend.name)

ibmq_qasm_simulator
